In [31]:
import pandas as pd
import numpy as np

In [5]:
y2018=pd.read_csv('synthetic_data_2018.csv')
y2019=pd.read_csv('synthetic_data_2019.csv')
y2020=pd.read_csv('synthetic_data_2020.csv')
y2021=pd.read_csv('synthetic_data_2021.csv')
y2022=pd.read_csv('synthetic_data_2022.csv')

In [33]:
data

,Unnamed: 0.1,Unnamed: 0,Classification,Disposition,datetime,Year,DayOfYear,Month,DayOfWeek,time_period,Building
0,0,0,Obstruction of Justice,Open Investigation,2019-02-13 13:05:00,2019,44,2,2,day_weekday,SRC MINI GRAND STAND
1,1,1,Theft,Open Investigation,2019-03-02 13:55:00,2019,61,3,5,weekend,PGP RESIDENCES (BLOCK 13)
2,2,2,Liquor Law Violation,Open Investigation,2019-03-18 03:05:00,2019,77,3,0,day_weekday,PGP RESIDENCES CANTEEN 2
3,3,3,Theft,Open Investigation,2018-11-16 16:42:00,2018,320,11,4,day_weekday,"KENT VALE APARTMENT BLK G, 115"
4,4,4,Assault,Open Investigation,2018-09-28 14:03:00,2018,271,9,4,day_weekday,YALE FITNESS CENTER
...,...,...,...,...,...,...,...,...,...,...,...
8665,1784,1784,Trespassing,Closed,2023-02-10 02:09:00,2023,41,2,4,day_weekday,LECTURE THEATRE 15
8666,1785,1785,Theft,Open Investigation,2023-06-06 22:10:00,2023,157,6,1,day_weekday,KUOK FOUNDATION HOUSE
8667,1786,1786,Theft: Bias Incident,Arrest,2023-04-06 08:14:00,2023,96,4,3,day_weekday,RAFFLES HALL (BLOCK D)
8668,1787,1787,Disorderly Conduct,Open Investigation,2023-10-29 22:16:00,2023,302,10,6,weekend,RAFFLES HALL (BLOCK D)


In [12]:
data=pd.concat([y2018,y2019,y2020,y2021,y2022],axis=0,ignore_index=True)

In [21]:
data['datetime']=pd.to_datetime(data['datetime'])

In [27]:
data['Year'] = data['datetime'].dt.year
data['DayOfYear'] = data['datetime'].dt.dayofyear
data['Month'] = data['datetime'].dt.month
data['DayOfWeek'] = data['datetime'].dt.weekday

daily_data_counts = data.groupby(['Year', 'DayOfYear']).size().reset_index(name='Incident Counts')
monthly_data_counts = data.groupby(['Year', 'Month']).size().reset_index(name='Incident Counts')
year_day_of_week_count = data.groupby(['Year','DayOfWeek']).size().reset_index(name='Incident Counts')
day_of_week_count = data.groupby(['DayOfWeek']).size().reset_index(name='Incident Counts')

In [24]:
locations = pd.read_csv("list_bldgs.csv")

In [28]:
def classify_time(row):
    if row.weekday() < 5 and 20 <= row.hour <= 8:
        return 'night_weekday'
    elif row.weekday() >= 5:
        return 'weekend'
    else:
        return 'day_weekday'

In [29]:
data['time_period'] = data['datetime'].apply(classify_time)

In [32]:
def assign_location(row):
    if row['time_period'] in ['night_weekday', 'weekend']:
        # 75% chance of being in a residence
        if np.random.rand() < 0.75:
            return locations[locations['Residence']]['Buildings'].sample(1).values[0]
        else:
            return locations[~locations['Residence']]['Buildings'].sample(1).values[0]
    else:
        # Random location
        return locations['Buildings'].sample(1).values[0]

data['Building'] = data.apply(assign_location, axis=1)

In [34]:
loc={'Location':[],'Longtitude':[],'Latitude':[]}
for bdg in data['Building']:
    id = locations.index[locations['Buildings'] == bdg][0]
    loc['Location'].append(list(locations['Location'])[id])
    loc['Longtitude'].append(list(locations['Longtitude'])[id])
    loc['Latitude'].append(list(locations['Latitude'])[id])

In [35]:
locs=pd.DataFrame(loc)

In [36]:
locs.head()

,Location,Longtitude,Latitude
0,YIH,1.298646,103.777740
1,PGP,1.290868,103.780276
2,PGP,1.290620,103.781695
3,KV,1.302494,103.770315
4,UTown,1.307479,103.772585


In [37]:
df=pd.concat([data,locs],axis=1)

In [38]:
df

,Unnamed: 0.1,Unnamed: 0,Classification,Disposition,datetime,Year,DayOfYear,Month,DayOfWeek,time_period,Building,Location,Longtitude,Latitude
0,0,0,Obstruction of Justice,Open Investigation,2019-02-13 13:05:00,2019,44,2,2,day_weekday,SRC MINI GRAND STAND,YIH,1.298646,103.777740
1,1,1,Theft,Open Investigation,2019-03-02 13:55:00,2019,61,3,5,weekend,PGP RESIDENCES (BLOCK 13),PGP,1.290868,103.780276
2,2,2,Liquor Law Violation,Open Investigation,2019-03-18 03:05:00,2019,77,3,0,day_weekday,PGP RESIDENCES CANTEEN 2,PGP,1.290620,103.781695
3,3,3,Theft,Open Investigation,2018-11-16 16:42:00,2018,320,11,4,day_weekday,"KENT VALE APARTMENT BLK G, 115",KV,1.302494,103.770315
4,4,4,Assault,Open Investigation,2018-09-28 14:03:00,2018,271,9,4,day_weekday,YALE FITNESS CENTER,UTown,1.307479,103.772585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8665,1784,1784,Trespassing,Closed,2023-02-10 02:09:00,2023,41,2,4,day_weekday,LECTURE THEATRE 15,SOC,1.295515,103.773452
8666,1785,1785,Theft,Open Investigation,2023-06-06 22:10:00,2023,157,6,1,day_weekday,KUOK FOUNDATION HOUSE,YIH,1.300351,103.773645
8667,1786,1786,Theft: Bias Incident,Arrest,2023-04-06 08:14:00,2023,96,4,3,day_weekday,RAFFLES HALL (BLOCK D),YIH,1.300302,103.772667
8668,1787,1787,Disorderly Conduct,Open Investigation,2023-10-29 22:16:00,2023,302,10,6,weekend,RAFFLES HALL (BLOCK D),YIH,1.300302,103.772667


In [47]:
df['Classification'][0]

'Obstruction of Justice'

In [40]:
temp=(df['Classification'].unique())

In [ ]:
LOST AND FOUND: 0.5 4335 (theft, tres
DAMAGED PROPERTY :0.25 2167.5 (Tresspassing, Harrassment,assault,fraud)
STOLEN ITEMS: 0.15 1300.5 (criminal mischief, shoplifting)
EMERGENCY INCIDENTS : 0.05 433.5 (Tresspass)
SEXUAL INCIDENTS :0.05 433.5 (Rape,stalking, disorderly conduct, lewdness)

In [44]:
for crime in temp:
  temp1=df[df['Classification'] == crime]
  print(crime,temp1.shape[0])

Obstruction of Justice 46
Theft 1829
Liquor Law Violation 69
Assault 209
False Public Alarm 239
Disorderly Conduct 142
Shoplifting 619
Harassment: Bias Incident 171
Trespass 473
Trespassing 1097
Identity Theft 50
Harassment 526
Lewdness 84
Harassment: Trespassing 52
Theft: Bias Incident 53
Extortion 125
Attempted Fraud 95
Criminal Mischief 688
Fraud 293
Harassment: Fraud 53
Attempted Trespass 2 43
Assault: Domestic Violence 65
Domestic Violence: Harassment 56
Bias Intimidation 61
Burglary 170
Defiant Trespass 109
Terroristic Threats 88
Theft: Criminal Mischief 107
Defiant Trespass: Aggravated Assault 65
Shoplifting  134
Attempted Extortion 115
Rape 211
Harassment: Domestic Violence 77
Possession of Burglary Tools 2 72
Hindering Possession of Burglary Tools 50
Stalking 63
Service of Warrant 61
Aggravated Assault 49
Attempted Theft 65
Intimidation 44
Harassment: Stalking 52


In [68]:
oth={'Incidents':[]}

In [69]:
for i in range(df.shape[0]):
    if df['Classification'][i] in ['Trespassing','Harassment','Assault','Fraud'] :
        oth['Incidents'].append('DAMAGED PROPERTY')
    elif df['Classification'][i] in ['Criminal Mischief','Shoplifting']:
        oth['Incidents'].append('STOLEN ITEMS')
    elif df['Classification'][i] in ['Trespass']:
        oth['Incidents'].append('EMERGENCY INCIDENTS')
    elif df['Classification'][i] in ['Rape','Stalking','Disorderly Conduct','Lewdness']:
        oth['Incidents'].append('SEXUAL INCIDENTS')
    else:
        oth['Incidents'].append('LOST AND FOUND')

# for st in oth['Incidents']:
#     if st=='LOST AND FOUND':
#         oth['Status']
    
    

In [70]:
oth1=pd.DataFrame(oth)

In [71]:
oth1

,Incidents
0,LOST AND FOUND
1,LOST AND FOUND
2,LOST AND FOUND
3,LOST AND FOUND
4,DAMAGED PROPERTY
...,...
8665,DAMAGED PROPERTY
8666,LOST AND FOUND
8667,LOST AND FOUND
8668,SEXUAL INCIDENTS


In [74]:
df1=pd.concat([df,oth1],axis=1)

In [75]:
df1.head()

,Unnamed: 0.1,Unnamed: 0,Classification,Disposition,datetime,Year,DayOfYear,Month,DayOfWeek,time_period,Building,Location,Longtitude,Latitude,Incidents
0,0,0,Obstruction of Justice,Open Investigation,2019-02-13 13:05:00,2019,44,2,2,day_weekday,SRC MINI GRAND STAND,YIH,1.298646,103.777740,LOST AND FOUND
1,1,1,Theft,Open Investigation,2019-03-02 13:55:00,2019,61,3,5,weekend,PGP RESIDENCES (BLOCK 13),PGP,1.290868,103.780276,LOST AND FOUND
2,2,2,Liquor Law Violation,Open Investigation,2019-03-18 03:05:00,2019,77,3,0,day_weekday,PGP RESIDENCES CANTEEN 2,PGP,1.290620,103.781695,LOST AND FOUND
3,3,3,Theft,Open Investigation,2018-11-16 16:42:00,2018,320,11,4,day_weekday,"KENT VALE APARTMENT BLK G, 115",KV,1.302494,103.770315,LOST AND FOUND
4,4,4,Assault,Open Investigation,2018-09-28 14:03:00,2018,271,9,4,day_weekday,YALE FITNESS CENTER,UTown,1.307479,103.772585,DAMAGED PROPERTY


In [76]:
df1.to_csv("data_except_priority&status.csv")